In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import io
import numpy as np

In [2]:
URLS = ['https://github.com/akutuzov/rushifteval_public/tree/main/durel/rushifteval1/data', 'https://github.com/akutuzov/rushifteval_public/tree/main/durel/rushifteval2/data', 'https://github.com/akutuzov/rushifteval_public/tree/main/durel/rushifteval3/data']
find_by_class_list = []
for i in URLS:
    page = requests.get(i)
    
    soup = BeautifulSoup( page.content , 'html.parser')
    find_by_class = soup.find_all('a', class_="Link--primary")
    find_by_class_list = find_by_class_list + find_by_class[3:]

In [3]:
judgements = []
uses = []
for i in find_by_class:
    judgements.append("https://raw.githubusercontent.com/"+i['href']+"/judgments.csv")
    uses.append("https://raw.githubusercontent.com/"+i['href']+"/uses.csv")

In [4]:
judgements_df = pd.DataFrame()
for i in judgements:
    judgements_df = pd.concat([judgements_df, pd.read_csv(io.StringIO(requests.get(i.replace("/tree","")).content.decode('utf-8')), delimiter='\t')])

In [5]:
uses_df = pd.DataFrame()
for i in uses:
    uses_df = pd.concat([uses_df, pd.read_csv(io.StringIO(requests.get(i.replace("/tree","")).content.decode('utf-8')), delimiter='\t')])

In [6]:
judgements_df.to_csv("judgements.csv", index=False)
uses_df.to_csv("uses.csv", index=False)

In [7]:
# Aggregate use pairs and extract median column
judgements_df = judgements_df.groupby(['identifier1', 'identifier2', 'lemma'])['judgment'].apply(list).reset_index(name='judgments')
# judgements_df['median_judgment'] = judgements_df['judgments'].apply(lambda x: np.nanmedian(list(x)))

# Remove pairs with nan median
# judgements_df = judgements_df[~judgements_df['median_judgment'].isnull()]
display(judgements_df)

,identifier1,identifier2,lemma,judgments
0,0.0,1.0,верховье,"[4.0, 4.0, 4.0]"
1,2.0,3.0,верховье,"[4.0, 4.0, 4.0]"
2,4.0,5.0,верховье,"[3.0, 4.0, 4.0]"
3,6.0,7.0,верховье,"[4.0, 4.0, 1.0]"
4,8.0,9.0,верховье,"[4.0, 4.0, 4.0]"
...,...,...,...,...
3302,6604.0,6605.0,ясли,"[4.0, 4.0, 4.0]"
3303,6606.0,6607.0,ясли,"[1.0, 1.0, 1.0]"
3304,6608.0,6609.0,ясли,"[4.0, 4.0, 4.0]"
3305,6610.0,6611.0,ясли,"[2.0, 1.0, 1.0]"


In [8]:
judgements_df.to_csv("judgements_grouped.csv", index=False)